In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
path = r'/Users/wangconghao/Documents/Computational Biology/Network_fusion/FYP_WULUE-master/TARGET_RNAseq/HTSeq_counts/'

In [3]:
clinical = pd.read_csv(path+'clinical.tsv', sep='\t', index_col='case_submitter_id', usecols=['case_submitter_id', 'vital_status'])
# clinical = pd.read_csv(path+'clinical.tsv', sep='\t', usecols=['case_id', 'vital_status'])
clinical.head()

,vital_status
case_submitter_id,
TARGET-30-PAKZRH,Alive
TARGET-30-PAPKXS,Dead
TARGET-30-PARFWB,Dead
TARGET-30-PAPUEB,Alive
TARGET-30-PAPVFD,Dead


In [8]:
clinical.shape

(155, 1)

In [4]:
clinical.replace({'Dead':1, 'Alive':0}, inplace=True)
clinical.head()

,vital_status
case_submitter_id,
TARGET-30-PAKZRH,0
TARGET-30-PAPKXS,1
TARGET-30-PARFWB,1
TARGET-30-PAPUEB,0
TARGET-30-PAPVFD,1


In [9]:
# sample_sheet = pd.read_csv(path+'sample_sheet.tsv', sep='\t', index_col='File ID', usecols=['File ID', 'File Name', 'Case ID'])
# sample_sheet = pd.read_csv(path+'sample_sheet.tsv', sep='\t', usecols=['File ID', 'File Name', 'Case ID'])
sample_sheet = pd.read_csv(path+'sample_sheet.csv')
sample_sheet.head()

,Case ID,File ID,File Name
0,TARGET-30-PAIFXV,5fe187f0-9749-48ca-96dc-7e4e97bab987,d5b38dfa-5b4b-4f52-b938-f3339b53f295.htseq_cou...
1,TARGET-30-PAIPGU,93da93a3-1ac3-4c40-8e18-b2930ccdea7d,8e5546e8-6043-4d8c-a926-9da125530883.htseq_cou...
2,TARGET-30-PAISNS,5fd90c0a-7ac4-49c5-b5f7-6772cb5bff10,c5af09bc-a060-4c6c-8076-f7d0b43f4822.htseq_cou...
3,TARGET-30-PAITCI,1449832f-64f6-4835-9880-4a8101dccb0b,d5912ef5-1d89-4809-951f-20c04bb25cf2.htseq_cou...
4,TARGET-30-PAIVHE,383a2d80-fb27-401e-ac65-70297065fc85,510dc7b2-f9b4-4ee3-8356-5f83afd67fbe.htseq_cou...


In [10]:
sample_sheet.shape

(157, 3)

In [11]:
# sample_sheet_copy = sample_sheet.copy()
# sample_sheet_copy.drop_duplicates(inplace=True)
# sample_sheet_copy.shape

In [12]:
samples = sample_sheet.to_numpy()
print(samples)

[['TARGET-30-PAIFXV' '5fe187f0-9749-48ca-96dc-7e4e97bab987'
  'd5b38dfa-5b4b-4f52-b938-f3339b53f295.htseq_counts.txt.gz']
 ['TARGET-30-PAIPGU' '93da93a3-1ac3-4c40-8e18-b2930ccdea7d'
  '8e5546e8-6043-4d8c-a926-9da125530883.htseq_counts.txt.gz']
 ['TARGET-30-PAISNS' '5fd90c0a-7ac4-49c5-b5f7-6772cb5bff10'
  'c5af09bc-a060-4c6c-8076-f7d0b43f4822.htseq_counts.txt.gz']
 ['TARGET-30-PAITCI' '1449832f-64f6-4835-9880-4a8101dccb0b'
  'd5912ef5-1d89-4809-951f-20c04bb25cf2.htseq_counts.txt.gz']
 ['TARGET-30-PAIVHE' '383a2d80-fb27-401e-ac65-70297065fc85'
  '510dc7b2-f9b4-4ee3-8356-5f83afd67fbe.htseq_counts.txt.gz']
 ['TARGET-30-PAKYZS' '1935ed8e-db54-4d72-8112-c6421f635958'
  'b791909c-76d4-46d8-8f2d-c3bc402357db.htseq_counts.txt.gz']
 ['TARGET-30-PAKZRH' 'ef30dd33-d892-408a-89b3-1392a9bfdc02'
  'dfe77aba-f62f-44e6-80ab-d4448109b835.htseq_counts.txt.gz']
 ['TARGET-30-PALBFW' 'fc0474e1-fb11-42b1-8fa5-227bba74948d'
  '1afe17a2-8c31-4dce-b8af-504da9baa93e.htseq_counts.txt.gz']
 ['TARGET-30-PALCBW' '7f

In [13]:
# cli_arr = clinical.to_numpy()
# print(cli_arr)

In [14]:
# manifest_txt = pd.read_table(path+'txt_manifest.txt', sep='\t', index_col='id', usecols=['id', 'filename'])
# manifest_txt = pd.read_table(path+'txt_manifest.txt', sep='\t', usecols=['id', 'filename'])
# manifest_txt.head()

In [15]:
# df = manifest_txt.loc['8be4dacc-7926-489c-9455-eca7f6f1b184']
# df

In [16]:
# manifest_txt_arr = manifest_txt.to_numpy()
# print(manifest_txt_arr)

In [22]:
def read_expression(file_id, filename):
    df = pd.read_table(path+'cases/'+os.path.join(file_id, filename), sep='\t', index_col=0, header=None, compression='gzip')
    array = df.to_numpy()
#     array = array.T[0]      # FPKM
    array = array.T[0][:-5] # counts
    return array

# a = read_expression('b19431df-0e34-4771-adda-6dbffcb60503','a46c4421-42d7-49b3-b109-f8bbe1e2bc83.htseq_counts.txt.gz')
# print(a)

In [23]:
l_exp = []
l_cli = []
for sample in samples:
    file_id = sample[1]
    filename = sample[2]
    case_id = sample[0]
#     print(id, filename)
    try:
        death = clinical.loc[case_id].to_numpy()
        expression = read_expression(file_id, filename)
#         temp = np.append(expression, death)
#         print(temp)
        l_exp.append(expression)
        l_cli.append(death)
    except:
        print('no such case:', case_id)

print(l_exp)
print(l_cli)

[array([1601,    9, 1587, ...,    0,    0,    0]), array([ 934,   10, 1414, ...,    0,    0,    0]), array([1799,    0, 1271, ...,    0,    0,    0]), array([1522,    0, 4287, ...,    0,    0,    0]), array([4609,    2, 1409, ...,    0,    0,    0]), array([1087,    0, 1230, ...,    0,    0,    0]), array([2829,   11, 2990, ...,    0,    0,    0]), array([1301,   11, 1267, ...,    0,    0,    0]), array([2290,    6, 1189, ...,    0,    0,    0]), array([1003,    3,  853, ...,    0,    0,    0]), array([885,   0, 540, ...,   0,   0,   0]), array([3018,   12, 2437, ...,    0,    0,    0]), array([1039,    2, 1359, ...,    0,    0,    0]), array([524,   5, 948, ...,   0,   0,   0]), array([ 841,    0, 1209, ...,    0,    0,    0]), array([1568,    0, 1333, ...,    0,    0,    0]), array([1039,    0, 2341, ...,    0,    0,    0]), array([1936,    2,  641, ...,    0,    0,    0]), array([1652,  178, 1623, ...,    0,    0,    0]), array([ 937,    1, 1659, ...,    0,    0,    0]), array([1871

In [24]:
exp = np.array(l_exp)
print(exp.shape)
cli = np.array(l_cli)
print(cli.shape)

(157, 60483)
(157, 1)


In [25]:
exp_df = pd.DataFrame(exp.T)
print(exp_df.shape)
exp_df.head()

(60483, 157)


,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,156
0,1601,934,1799,1522,4609,1087,2829,1301,2290,1003,...,1235,1235,956,786,956,786,1023,783,1023,783
1,9,10,0,0,2,0,11,11,6,3,...,3,3,1,6,1,6,4,3,4,3
2,1587,1414,1271,4287,1409,1230,2990,1267,1189,853,...,2519,2519,754,1188,754,1188,1030,784,1030,784
3,674,424,573,682,1089,856,1286,936,957,397,...,1622,1622,418,572,418,572,801,1233,801,1233
4,202,281,431,310,557,227,538,239,235,154,...,865,865,192,323,192,323,212,273,212,273


In [26]:
m = dict()
c = 1
for i in exp_df.columns:
	if i not in m:
		m[i] = 'F'+str(c)
		c+=1

print(m)

{0: 'F1', 1: 'F2', 2: 'F3', 3: 'F4', 4: 'F5', 5: 'F6', 6: 'F7', 7: 'F8', 8: 'F9', 9: 'F10', 10: 'F11', 11: 'F12', 12: 'F13', 13: 'F14', 14: 'F15', 15: 'F16', 16: 'F17', 17: 'F18', 18: 'F19', 19: 'F20', 20: 'F21', 21: 'F22', 22: 'F23', 23: 'F24', 24: 'F25', 25: 'F26', 26: 'F27', 27: 'F28', 28: 'F29', 29: 'F30', 30: 'F31', 31: 'F32', 32: 'F33', 33: 'F34', 34: 'F35', 35: 'F36', 36: 'F37', 37: 'F38', 38: 'F39', 39: 'F40', 40: 'F41', 41: 'F42', 42: 'F43', 43: 'F44', 44: 'F45', 45: 'F46', 46: 'F47', 47: 'F48', 48: 'F49', 49: 'F50', 50: 'F51', 51: 'F52', 52: 'F53', 53: 'F54', 54: 'F55', 55: 'F56', 56: 'F57', 57: 'F58', 58: 'F59', 59: 'F60', 60: 'F61', 61: 'F62', 62: 'F63', 63: 'F64', 64: 'F65', 65: 'F66', 66: 'F67', 67: 'F68', 68: 'F69', 69: 'F70', 70: 'F71', 71: 'F72', 72: 'F73', 73: 'F74', 74: 'F75', 75: 'F76', 76: 'F77', 77: 'F78', 78: 'F79', 79: 'F80', 80: 'F81', 81: 'F82', 82: 'F83', 83: 'F84', 84: 'F85', 85: 'F86', 86: 'F87', 87: 'F88', 88: 'F89', 89: 'F90', 90: 'F91', 91: 'F92', 92: 'F

In [27]:
exp_df.rename(columns= lambda x:m[x], inplace=True)
exp_df.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F148,F149,F150,F151,F152,F153,F154,F155,F156,F157
0,1601,934,1799,1522,4609,1087,2829,1301,2290,1003,...,1235,1235,956,786,956,786,1023,783,1023,783
1,9,10,0,0,2,0,11,11,6,3,...,3,3,1,6,1,6,4,3,4,3
2,1587,1414,1271,4287,1409,1230,2990,1267,1189,853,...,2519,2519,754,1188,754,1188,1030,784,1030,784
3,674,424,573,682,1089,856,1286,936,957,397,...,1622,1622,418,572,418,572,801,1233,801,1233
4,202,281,431,310,557,227,538,239,235,154,...,865,865,192,323,192,323,212,273,212,273


In [28]:
exp_df.to_csv(path+'DataMat.csv')

In [34]:
cli_df = pd.DataFrame(cli)
print(cli_df.shape)
cli_df.head()

(157, 1)


,0
0,1
1,1
2,1
3,1
4,1


In [35]:
cli_df.to_csv(path+'ClinicalData.csv')

In [45]:
array_dataset = pd.read_csv(path+"DataMat.csv")
clinical_dataset = pd.read_csv(path+"ClinicalData.csv")

In [50]:
X = array_dataset.iloc[:, 1:].values
Y = clinical_dataset.iloc[:, :].values

In [51]:
print(X)
print(Y)

[[    1172     2076      712 ...      885     3049     2708]
 [      72        3        8 ...        0        4       10]
 [    2264     1812     2997 ...      540     2029     2348]
 ...
 [       0        0        0 ...        0        0        0]
 [       0        0        0 ...        0        0        0]
 [21768857 24500254 21264151 ... 13438747 15752593 15248373]]
[[  0   1]
 [  1   1]
 [  2   0]
 [  3   0]
 [  4   0]
 [  5   0]
 [  6   1]
 [  7   1]
 [  8   0]
 [  9   1]
 [ 10   0]
 [ 11   1]
 [ 12   0]
 [ 13   0]
 [ 14   1]
 [ 15   1]
 [ 16   0]
 [ 17   0]
 [ 18   1]
 [ 19   1]
 [ 20   1]
 [ 21   1]
 [ 22   1]
 [ 23   0]
 [ 24   1]
 [ 25   1]
 [ 26   1]
 [ 27   1]
 [ 28   1]
 [ 29   0]
 [ 30   0]
 [ 31   1]
 [ 32   1]
 [ 33   0]
 [ 34   0]
 [ 35   1]
 [ 36   0]
 [ 37   0]
 [ 38   0]
 [ 39   1]
 [ 40   1]
 [ 41   0]
 [ 42   1]
 [ 43   1]
 [ 44   1]
 [ 45   0]
 [ 46   1]
 [ 47   0]
 [ 48   1]
 [ 49   0]
 [ 50   0]
 [ 51   1]
 [ 52   0]
 [ 53   1]
 [ 54   0]
 [ 55   1]
 [ 56   1]
 

In [25]:
# df = pd.DataFrame(l)
# df.head()

,0,1,2,3,4,5,6,7,8,9,...,60479,60480,60481,60482,60483,60484,60485,60486,60487,60488
0,1172,72,2264,1577,1317,2113,3466,1941,1631,3628,...,0,0,0,0,12467072,3662051,0,0,21768857,1
1,2076,3,1812,1139,652,243,1609,1910,1991,1943,...,0,0,0,0,10315094,3962382,0,0,24500254,1
2,712,8,2997,1841,1143,575,1390,1352,1730,3054,...,0,0,0,0,17382334,3498292,0,0,21264151,0
3,3522,9,1776,2347,1156,802,1915,2052,2499,2613,...,0,0,0,0,14590902,3229494,0,0,12206783,0
4,1184,2,1188,808,438,183,1241,1558,901,1193,...,0,0,0,0,3450124,1998277,0,0,7115848,0


In [26]:
# df.to_csv(path+'data.csv', index=False, header=False)

In [31]:
# data = pd.read_csv(path+'data.csv', header=None)
# data.head()

,0,1,2,3,4,5,6,7,8,9,...,60479,60480,60481,60482,60483,60484,60485,60486,60487,60488
0,1172,72,2264,1577,1317,2113,3466,1941,1631,3628,...,0,0,0,0,12467072,3662051,0,0,21768857,1
1,2076,3,1812,1139,652,243,1609,1910,1991,1943,...,0,0,0,0,10315094,3962382,0,0,24500254,1
2,712,8,2997,1841,1143,575,1390,1352,1730,3054,...,0,0,0,0,17382334,3498292,0,0,21264151,0
3,3522,9,1776,2347,1156,802,1915,2052,2499,2613,...,0,0,0,0,14590902,3229494,0,0,12206783,0
4,1184,2,1188,808,438,183,1241,1558,901,1193,...,0,0,0,0,3450124,1998277,0,0,7115848,0


In [32]:
# x = data.iloc[:, :-1].values
# y = data.iloc[:, -1].values

In [33]:
# print(x)

[[    1172       72     2264 ...        0        0 21768857]
 [    2076        3     1812 ...        0        0 24500254]
 [     712        8     2997 ...        0        0 21264151]
 ...
 [     885        0      540 ...        0        0 13438747]
 [    3049        4     2029 ...        0        0 15752593]
 [    2708       10     2348 ...        0        0 15248373]]


In [34]:
# print(y)

[1 1 0 0 0 0 1 1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 0 1 0
 0 0 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 0 1 1 1 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 1
 0 1 0 1 0 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 1 1 0 1 1 1 0 0 1 1 0 0 0 1 0 1 1
 1 0 0 1 1 1 0 1 0 1 0 1 0 0 0 1 1 0 1 1 0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 0 0
 1 0 0 0 0 0 1 0 1 1 0 0 0]
